In [1]:
import numpy    as np
import warnings #------------------------------------------------ To suppress warnings
import h5py #---------------------------------------------------- To read hdf5 files
import pandas   as pd
import matplotlib.pyplot as plt
import glob
import xesmf as xe
import xarray as xr
from   ast                           import literal_eval #----------------------------------- For literal evaluation of a string to extract python objects
from   pyproj                        import Proj, transform #----------------------------- To interconvert different projections
from   netCDF4                       import Dataset #------------------------------------ To read nc , nc4 and hdf4 files
from   IPython.core.interactiveshell import InteractiveShell  #---- Output all jupyter lab inputs instead of the last one
InteractiveShell.ast_node_interactivity = "all"
warnings.simplefilter('ignore','always')

/home/kaushal/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
/home/kaushal/anaconda3/lib/python3.7/site-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [2]:
in_proj  = Proj('+proj=sinu +R=6371007.181 +nadgrids=@null +wktext') #------ Specify input projection
out_proj = Proj(init='epsg:4326') #----------------------------------------- Specify output projection
data     = np.genfromtxt('sn_bound_10deg.txt', skip_header = 7, skip_footer = 3)

def extract_pixel_coordinates(ULx,Uly,LRx,LRy,shape):
    x        = np.linspace(ULx, LRx, shape[0], endpoint=False) + abs((ULx-LRx)/(2*shape[0]))
    y        = np.linspace(ULy, LRy, shape[0], endpoint=False) - abs((ULy-LRy)/(2*shape[0]))
    xx, yy   = np.meshgrid(x,y)
    xs       = xx.flatten()
    ys       = yy.flatten()
    plon, plat = transform(in_proj, out_proj, xs, ys)
    return plon, plat

In [3]:
par             = Dataset('MCD18A2/121/MCD18A2.A2018121.h11v04.006.2019091181205.hdf', mode='r')
gmt_1500        = par.variables['GMT_1500_PAR'][:]
struct          = getattr(par, 'StructMetadata.0')
struct1         = struct[struct.find('UpperLeftPointMtrs'): struct.find('LowerRightMtrs')][19:-3] #-------------]
struct2         = struct[struct.find('LowerRightMtrs')    : struct.find('Projection')    ][15:-3] #-------------] Extract upper right and lower left
ULx, ULy        = literal_eval(struct1) #-----------------------------------------------------------------------] coordinates of the tile of opened par file
LRx, LRy        = literal_eval(struct2) #-----------------------------------------------------------------------]
par_lon,par_lat = extract_pixel_coordinates(ULx,ULy,LRx,LRy,(240,240)) #----------- Extract the par lat,lon meshgrid in proper projection

In [4]:
par_lon = par_lon.reshape(240,240)
par_lat = par_lat.reshape(240,240)

In [7]:
%matplotlib
plt.scatter(par_lon,par_lat, s=0.5)

Using matplotlib backend: Qt5Agg


In [23]:
gmt_1500.shape
par_lon.shape
par_lat.shape

(240, 240)

(57600,)

(57600,)

In [24]:
tar_lon  = par_lon [1000:2000]
tar_lat  = par_lat [1000:2000]

In [25]:
par_lon.max(), par_lon.min()
par_lat.max(), par_lat.min()

(-78.37555130561373, -108.82110827388532)

(49.9791666621754, 40.020833329742324)

In [26]:
tar_lon = np.linspace(-110,-70, 1000)#.reshape(50,20)
tar_lat = np.linspace(40  , 49, 1000)#.reshape(50,20)

In [27]:
grid_in  = {'lon': par_lon, 'lat': par_lat }
grid_out = {'lon': tar_lon, 'lat': tar_lat }

In [28]:
regridder = xe.Regridder(grid_in, grid_out, 'bilinear')
regridder.clean_weight_file()
regridder

MemoryError: 

In [ ]:
data_out = regridder(original)  # regrid a basic numpy array
data_out.shape

In [ ]:
data = np.arange(20).reshape(4, 5)
plt.pcolormesh(data)

In [ ]:
data.shape

In [ ]:
grid_in = {'lon': np.linspace(0, 40, 5),
           'lat': np.linspace(0, 20, 4)
          }

# output grid has a larger coverage and finer resolution
grid_out = {'lon': np.linspace(-20, 60, 51),
            'lat': np.linspace(-10, 30, 41)
           }

grid_in
grid_out

In [ ]:
regridder = xe.Regridder(grid_in, grid_out, 'bilinear')
print('1')
#regridder.clean_weight_file()
print('2')
regridder

In [ ]:
ds = xr.tutorial.open_dataset('air_temperature') # use xr.tutorial.load_dataset() for xarray<v0.11.0
ds

In [ ]:
dr = ds['air']

In [ ]:
ds['lat'].values, ds['lon'].values

In [ ]:

ds_out = xr.Dataset({'lat': (['lat'], np.arange(15, 75.01, 0.25)),
                     'lon': (['lon'], np.arange(200, 330.01, 0.5)),
                    }
                   )
ds_out

In [ ]:
regridder = xe.Regridder(ds, ds_out, 'bilinear')
regridder  # print basic regridder information.